In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, date
from math import *
import json
# %matplotlib inline
from decimal import *
# getcontext().prec = 6
# import re
# from decimal import Decimal

# from os import listdir
# from os.path import isfile, join

import pandas_ta as ta

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib
# from mplfinance import candlestick_ohlc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates



import numpy as np

from os.path import exists

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
%matplotlib inline
from decimal import *
getcontext().prec = 6
import seaborn as sns

matplotlib.rcParams.update({'font.size': 5, 'lines.linewidth': 0.5, 'figure.dpi': 300})


In [16]:
def LoadCandles(pair):
    fld = '/media/grenada/Data/Crypto-Data-Feed/freq-user-data/data/binance'    
    return pd.read_json(f'{fld}/{pair}.json'
    ).set_axis(['ts','open','high','low','close','volume'],axis=1
    ).assign(ts=lambda x: pd.to_datetime(x['ts'], unit='ms', utc=False)
    ).set_index('ts').apply(np.log)

# df

In [4]:
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise

def pos_vel_filter(x, P, R, Q=0., dt=1.0):
    """ Returns a KalmanFilter which implements a
    constant velocity model for a state [x dx].T
    """
    
    kf = KalmanFilter(dim_x=2, dim_z=1)
    kf.x = np.array([x[0], x[1]]) # location and velocity
    kf.F = np.array([[1., dt],
                     [0.,  1.]])  # state transition matrix
    kf.H = np.array([[1., 0]])    # Measurement function
    kf.R *= R                     # measurement uncertainty
    if np.isscalar(P):
        kf.P *= P                 # covariance matrix 
    else:
        kf.P[:] = P               # [:] makes deep copy
    if np.isscalar(Q):
        kf.Q = Q_discrete_white_noise(dim=2, dt=dt, var=Q)
    else:
        kf.Q[:] = Q
    return kf

In [5]:
def run(x0=(0.,0.), P=500, R=0, Q=0, dt=1.0, 
        zs=None,
        ):

    kf = pos_vel_filter(x0, R=R, P=P, Q=Q, dt=dt)  

    # run the kalman filter and store the results
    xs, cov = [], []
    for z in zs:
        kf.predict()
        kf.update(z)
        xs.append(kf.x)
        cov.append(kf.P)

    xs, cov = np.array(xs), np.array(cov)
    return xs, cov

In [18]:
# zs = [[x] for x in df.close.values]
# Ms, Ps = run(zs=zs, R=10, Q=0.01, P=P)
df = LoadCandles('AVAX_USDT-1d')
df.head(3)

,open,high,low,close,volume
ts,,,,,
2020-09-22,-0.162519,1.945910,-0.162519,1.671342,17.263651
2020-09-23,1.672957,1.678964,1.223775,1.262713,16.350842
2020-09-24,1.261440,1.586640,1.224746,1.534951,15.776710


In [20]:
wdf = df#.head(900)
sl_dt= FloatSlider(description="dt2", min=0, max=0.01, step=0.0004, value=0.001)
sl_q = FloatSlider(description="q", min=0, max=0.00001, step=0.000001, value=0)
sl_r = FloatSlider(description="r", min=0, max=50, step=5, value=5)

def printer(dt,q,r):
    print(dt,q,r)
    P = np.diag([10,10])
    Ms, Ps = run(zs=wdf.close.values, R=r, Q=q, P=P,dt=dt)
    [xs,vs] = Ms.T

    fig = plt.figure(constrained_layout=True,figsize=(12,5))
    (ax,ax1) = fig.subplots(2,1,height_ratios=[3,1],sharex=True)
    wdf.close.plot(ax=ax)
    ax.plot(wdf.index, xs)
    ax1.plot(wdf.index, vs)
    ax1.axhline()
    ax1.set_ylim([-0.02,0.02])

    

ui = widgets.HBox([sl_dt, sl_q, sl_r])
out = widgets.interactive_output(printer, {'dt': sl_dt, 'q':sl_q, 'r': sl_r})
display(ui, out)

Output()